# Advanced matplotlib

Pierre Augier (LEGI), Cyrille Bonamy (LEGI), Eric Maldonado (Irstea), Franck Thollard (ISTerre), Christophe Picard (LJK), Loïc Huder (ISTerre)

## Introduction
This is the second part of the introductive presentation given in the [Python initiation training](https://gricad-gitlab.univ-grenoble-alpes.fr/python-uga/py-training-2017/blob/master/ipynb/pres111_intro_matplotlib.ipynb). 

The aim is to present more advanced usecases of matplotlib.

## Quick reminders

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
X = np.arange(0, 2, 0.01)
Y = np.exp(X) - 1

plt.plot(X, X, linewidth=3)
plt.plot(X, Y)
plt.plot(X, X**2)
plt.xlabel('Abscisse')
plt.ylabel('Ordinate')

## Object-oriented plots

While doing the job, the previous example does not allow to unveil the power of matplotlib. For that, we need to keep in mind that in matplotlib plots, **everything** is an object.
<img src='images/anatomy.png' alt='https://matplotlib.org/gallery/showcase/anatomy.html' />

It is therefore possible to change any aspect of the figure by acting on the appropriate objects. 

### The same example with objects

In [ ]:
fig = plt.figure()
print('Fig is an instance of', type(fig))
ax = fig.add_subplot(111) # More on subplots later...
print('Ax is an instance of', type(ax))

X = np.arange(0, 2, 0.01)
Y = np.exp(X) - 1

# Storing results of the plot
l1 = ax.plot(X, X, linewidth=3, label='Linear')
l2 = ax.plot(X, X**2, label='Square')
l3 = ax.plot(X, Y, label='$y = e^{x} - 1$')

xlab = ax.set_xlabel('Abscissa')
ylab = ax.set_ylabel('Ordinate')

ax.set_xlim(0, 2)

ax.legend()

In [ ]:
# ax.plot returns in fact a list of the lines plotted by the instruction
print(type(l3))
# In this case, we plotted the lines one by one so l3 contains only the line corresponding to the exp function
exp_line = l3[0]
print(type(exp_line))

This way, we can have access to the `Line2D` objects and therefore to all their attributes (and change them!). This includes:
- **get_data/set_data**: to get/set the numerical xdata, ydata of the line
- **get_color/set_color**: to get/set the color of the line
- **get_marker/set_marker**: to get/set the markers
- ...

See https://matplotlib.org/api/_as_gen/matplotlib.lines.Line2D.html for the complete list.

_Note: `Line2D` is based on the `Artist` class from which any graphical element inherits (lines, ticks, axes...)._

### An application: ticks

A common manipulation when designing figures for articles is to change the ticks location. Matplotlib provides several ways to do this

#### Direct manipulation

In [ ]:
from calendar import day_name

weekdays = list(day_name)
print(weekdays)
temperatures = [20., 22., 16., 18., 17., 19., 20.]

fig, ax = plt.subplots()

ax.plot(temperatures, marker='o', markersize=10)
ax.set_xlabel('Weekday')
ax.set_ylabel('Temperature ($^{\circ}C$)')

In [ ]:
# Change locations
ax.set_yticks(np.arange(15, 25, 0.5))

# Change locations AND labels
ax.set_xticks(range(7))
ax.set_xticklabels(weekdays)

ax.set_xlabel('')

# Show the updated figure
fig

#### With `Locators`
`Locators` are objects that give rules to generate the tick locations. See https://matplotlib.org/api/ticker_api.html.
<img src='images/tick_locators.png' alt='https://matplotlib.org/gallery/ticks_and_spines/tick-locators.html' />

For example, for the yticks in the previous example, we could have done

In [ ]:
import matplotlib.ticker as ticker

# Change locator for the major ticks of yaxis
ax.yaxis.set_major_locator(ticker.MultipleLocator(0.5))

fig

#### Major and minor ticks
matplotlib provides two types of ticks: major and minor. The parameters and aspect of the two kinds can be handled separately.

In [ ]:
import matplotlib.ticker as ticker

# Change locator for the major ticks of yaxis
ax.yaxis.set_major_locator(ticker.MultipleLocator(1.))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.5))

fig

### An application: subplots

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, sharey=True)
"""
# Equivalent to
fig = plt.figure()
axes = []
axes.append(fig.add_subplot(131))
axes.append(fig.add_subplot(132, sharey=axes[0]))
axes.append(fig.add_subplot(133, sharey=axes[0]))
"""

# This is only to have the same colors as before
color_cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']

X = np.arange(0, 2, 0.01)
Y = np.exp(X) - 1

axes[0].set_title('Linear')
axes[0].plot(X, X, linewidth=3, color=color_cycle[0])
axes[1].set_title('Square')
axes[1].plot(X, X**2, label='Square', color=color_cycle[1])
axes[2].set_title('$y = e^{x} - 1$')
axes[2].plot(X, Y, label='$y = e^{x} - 1$', color=color_cycle[2])

axes[0].set_ylabel('Ordinate')
for ax in axes:
    ax.set_xlabel('Abscissa')
    ax.set_xlim(0, 2)

### Fancier subplots with gridspec

In [ ]:
import matplotlib.gridspec as gridspec

fig = plt.figure()
gs = gridspec.GridSpec(2, 2, figure=fig) # 2 rows and 2 columns
X = np.arange(-3, 3, 0.01)*np.pi
ax1 = fig.add_subplot(gs[0,0]) # 1st row, 1st column
ax2 = fig.add_subplot(gs[1,0]) # 2nd row, 1st column
ax3 = fig.add_subplot(gs[:,1]) # all rows, 2nd column
ax1.plot(X, np.cos(2*X), color="red")
ax2.plot(X, np.sin(2*X), color="magenta")
ax3.plot(X, X**2)

## Interactivity

## Animations

From the matplotlib page (https://matplotlib.org/api/animation_api.html):
> The easiest way to make a live animation in matplotlib is to use one of the Animation classes.
><table>
    <tr><td>FuncAnimation</td><td>Makes an animation by repeatedly calling a function func.</td></tr>
    <tr><td>ArtistAnimation</td><td>Animation using a fixed set of Artist objects.</td></tr>
</table>

### Example from matplotlib page
This example uses `FuncAnimation` to animate the plot of a sin function.

The animation consists in making repeated calls to the `update` function that adds at each frame a datapoint to the plot.

In [ ]:
from matplotlib import animation

fig, ax = plt.subplots()
xdata, ydata = [], []
ln, = plt.plot([], [], 'ro')

def init():
    ax.set_xlim(0, 2*np.pi)
    ax.set_ylim(-1, 1)
    return ln,

def update(frame):
    xdata.append(frame)
    ydata.append(np.sin(frame))
    ln.set_data(xdata, ydata)
    return ln,

ani = animation.FuncAnimation(fig, update, frames=np.linspace(0, 2*np.pi, 128),
                    init_func=init, blit=True)
plt.show()

The previous code executed in a regular Python script should display the animation without problem. In a Jupyter Notebook, it is necessary to use IPython to display it in HTML.

In [ ]:
from IPython.display import HTML

HTML(ani.to_jshtml())

### Stroop test
The [Stroop effect](http://en.wikipedia.org/wiki/Stroop_effect) is when a psychological cause inteferes with the reaction time of a task.

A common demonstration of this effect (called a Stroop test) is naming the color in which a word is written if the word describes another color. This usually takes longer than for a word that is not a color.

Ex: Naming blue for <div style='text-align:center; font-size:36px'><span style='color:blue'>RED</span> vs. <span style='color:blue'>BIRD</span></div>

_Funfact: As this test relies on the significance of the words, people that are more used to English should find the test more difficult !_

In this part, we show how `matplotlib` animations can generate a Stroop test that shows random color words in random colors at random positions. The person passing the test should then name the color in which the word is written.

#### With `FuncAnimation`
We will generate a single object `word` whose position, color and text will be updated by the repeatedly called function.

In [ ]:
import random

def generate_random_colored_word(words, colors):
    displayed_text = random.choice(words).upper()
    text_color = random.choice(colors)
    xy_position = (random.random(), random.random())
    return xy_position, displayed_text, text_color


def update(frame):
    xy_position, displayed_text, text_color = generate_random_colored_word(wordset, colorset)
    word.set_position(xy_position)
    word.set_color(text_color)
    word.set_text(displayed_text)
    return word

fig, ax = plt.subplots()

colorset = ['red', 'blue', 'yellow', 'green', 'purple']
wordset = colorset

xy_position, displayed_text, text_color = generate_random_colored_word(wordset, colorset)
word = ax.annotate(displayed_text, xy_position, xycoords='axes fraction', color=text_color, size=36)

ani = animation.FuncAnimation(fig, update, interval=1000)
plt.show()

In [ ]:
from IPython.display import HTML

HTML(ani.to_jshtml())

#### With `ArtistAnimation`
Rather than updating through a function, `ArtistAnimation` requires to generate first all the `Artists` that will be displayed during the whole animation. 

A list of `Artists` must therefore be supplied for each frame. Then, all frame lists must be compiled in a single list (of lists) that will be given in argument of `ArtistAnimation`.

In our case, to reproduce the behaviour above, we need to have only one word per frame. Each frame will therefore have a list of a single element (the colored word for this frame).

In [ ]:
fig, ax = plt.subplots()

N_frames = 200
words = []

colorset = ['red', 'blue', 'yellow', 'green', 'purple']
wordset = colorset

# Generate the list of lists of Artists.
for i in range(N_frames):
    xy_position, displayed_text, text_color = generate_random_colored_word(wordset, colorset)
    # The list of the frame contains only a single word
    frame_artists = [ax.annotate(displayed_text, xy_position, xycoords='axes fraction', color=text_color, size=36)]
    words.append(frame_artists)

ani = animation.ArtistAnimation(fig, words, interval=1000)
plt.show()

In [ ]:
from IPython.display import HTML

HTML(ani.to_jshtml())

#### Example with multiple `Artists`: two words at once from two wordsets !

In [ ]:
# We can remove the axes for a cleaner test
fig = plt.figure()
ax = fig.add_subplot(111, frameon=False)
ax.set_xticks([])
ax.set_yticks([])

N_frames = 200
words = []

colorset = ['red', 'blue', 'yellow', 'green', 'purple']
wordset = colorset
wordset2 = ['bed', 'glue', 'mellow', 'grain', 'people']

# Generate the list of lists of Artists.
for i in range(N_frames):
    xy_position, displayed_text, text_color = generate_random_colored_word(wordset, colorset)
    xy_position2, displayed_text2, text_color2 = generate_random_colored_word(wordset2, colorset)
    # The list of the frame contains only a single word
    frame_artists = [ax.annotate(displayed_text, xy_position, xycoords='axes fraction', color=text_color, size=36),
                     ax.annotate(displayed_text2, xy_position2, xycoords='axes fraction', color=text_color2, size=36)]
    words.append(frame_artists)

ani = animation.ArtistAnimation(fig, words, interval=1000)
plt.show()

In [ ]:
from IPython.display import HTML

HTML(ani.to_jshtml())